<a href="https://colab.research.google.com/github/Kkordik/NovelQSI/blob/main/Compare_Base_with_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 976.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
import datasets
from evaluate import evaluator
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load the dataset
dataset_name = "Kkordik/NovelQSI" # @param {type:"string"}
hf_model_name = "mrm8488/longformer-base-4096-finetuned-squadv2" # @param {type:"string"}

dataset = datasets.load_dataset(dataset_name, split="test")

# Load the model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(hf_model_name)
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
task_evaluator = evaluator("question-answering")

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/longformer-base-4096-finetuned-squadv2 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [34]:
filtered_dataset = dataset.filter(lambda row: 2100 <= len(tokenizer.tokenize(row["context"] + " " + row["question"])) <= 2300)
print(filtered_dataset)

Filter:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'answers', 'id'],
    num_rows: 231
})


In [35]:
tokenizer.model_max_length = 2500

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data=filtered_dataset,
    strategy="bootstrap",
    n_resamples=30,
)
print(eval_results)

Filter:   0%|          | 0/231 [00:00<?, ? examples/s]

{'exact_match': {'confidence_interval': (8.754452551305853, 14.718614718614718), 'standard_error': 1.8579217243778676, 'score': 12.121212121212121}, 'f1': {'confidence_interval': (18.469101076147584, 28.28409063313956), 'standard_error': 2.896728175757627, 'score': 22.799422799422796}, 'total_time_in_seconds': 178.5769887460001, 'samples_per_second': 1.2935597224598967, 'latency_in_seconds': 0.7730605573419919}


In [37]:
fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained("Kkordik/test_longformer_4096_qsi")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("Kkordik/test_longformer_4096_qsi")

In [38]:
fine_tuned_tokenizer.model_max_length = 2500

eval_results = task_evaluator.compute(
    model_or_pipeline=fine_tuned_model,
    tokenizer=fine_tuned_tokenizer,
    data=filtered_dataset,
    strategy="bootstrap",
    n_resamples=30,
)
print(eval_results)

Filter:   0%|          | 0/231 [00:00<?, ? examples/s]

{'exact_match': {'confidence_interval': (16.017316017316016, 24.242424242424242), 'standard_error': 2.9434375492784994, 'score': 20.346320346320347}, 'f1': {'confidence_interval': (23.123469058324783, 31.823648733317036), 'standard_error': 2.593030474995015, 'score': 26.580086580086572}, 'total_time_in_seconds': 186.96806366500005, 'samples_per_second': 1.235505120349827, 'latency_in_seconds': 0.8093855569913422}
